In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, pipeline
import torch
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, RobertaTokenizer
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

2024-05-01 00:05:41.586103: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 00:05:41.586166: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 00:05:41.586251: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-01 00:05:41.601476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 00:05:42.228570: W tensorflow/compiler/

In [2]:
stop_words = stopwords.words('english')
test_df = pd.read_csv('../data/csv/test_Anon.csv')

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [5]:
test_df['text'] = test_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(encode, batched=True)

Map:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("../models/ECHR_Anon/distilbert_echr_model/removed_stopwords/accuracy/checkpoint-888", num_labels=2)
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

In [9]:
results = pipe(tokenizer.decode(case, clean_up_tokenization_spaces=True, skip_special_tokens=True) for case in test_dataset['input_ids'])

In [10]:
predictions = list(result['label'] for result in results)

In [11]:
report = classification_report(list('NON_VIOLATED' if outcome==0 else 'VIOLATED' for outcome in test_dataset['labels']), predictions)

In [12]:
print(report)

              precision    recall  f1-score   support

NON_VIOLATED       0.73      0.62      0.67      1024
    VIOLATED       0.82      0.88      0.85      1974

    accuracy                           0.79      2998
   macro avg       0.77      0.75      0.76      2998
weighted avg       0.79      0.79      0.79      2998

